In [ ]:
# import modules
import uproot, sys, time, math, pickle, os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import awkward as ak
from tqdm import tqdm
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker
from scipy.special import betainc
from scipy.stats import norm

# import config functions
from jet_faking_plot_config import getWeight, zbi, sample_dict, getVarDict
from plot_var import variables, variables_data, ntuple_names, ntuple_names_BDT
from n_1_iteration_functions import get_best_cut, calculate_significance, apply_cut_to_fb, apply_all_cuts, compute_total_significance, n_minus_1_optimizer
# from cut_config import cut_config

# Set up plot defaults
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 14.0,10.0  # Roughly 11 cm wde by 8 cm high  
mpl.rcParams['font.size'] = 20.0 # Use 14 point font
sns.set(style="whitegrid")

font_size = {
    "xlabel": 17,
    "ylabel": 17,
    "xticks": 15,
    "yticks": 15,
    "legend": 14
}

plt.rcParams.update({
    "axes.labelsize": font_size["xlabel"],  # X and Y axis labels
    "xtick.labelsize": font_size["xticks"],  # X ticks
    "ytick.labelsize": font_size["yticks"],  # Y ticks
    "legend.fontsize": font_size["legend"]  # Legend
})

tot = []
data = pd.DataFrame()
unweighted_bcut, weighted_bcut, unweighted_acut, weighted_acut = [], [], [], []
ntuple_names = ['ggHyyd','Zjets','Zgamma','Wgamma','Wjets','gammajet_direct', 'data23']

def test(fb):
    # checking if there are any none values
    mask = ak.is_none(fb['met_tst_et'])
    n_none = ak.sum(mask)
    print("Number of none values: ", n_none)
    # if n_none > 0:
    #     fb = fb[~mask]
    # print("Events after removing none values: ", len(fb), ak.sum(ak.is_none(fb['met_tst_et'])))

def print_cut(ntuple_name, fb, label):
    print(f"Unweighted Events {label}: ", len(fb))
    if ntuple_name == 'data23':
        print(f"Weighted Events {label}: ", sum(getWeight(fb, ntuple_name, jet_faking=True)))
    else: 
        print(f"Weighted Events {label}: ", sum(getWeight(fb, ntuple_name)))

def check_memory(prefix=""):
    mem = psutil.virtual_memory()
    print(f"{prefix}Available RAM: {mem.available / (1024**3):.2f} GB "
          f"({mem.percent}% used)")

for i in range(len(ntuple_names)):
    ucut, wcut = [], []
    start_time = time.time()
    ntuple_name = ntuple_names[i]
    if ntuple_name == 'data23': # data
        path = f"/data/fpiazza/ggHyyd/Ntuples/MC23d/withVertexBDT/data23_y_BDT_score.root" 
        print('processing file: ', path)
        f = uproot.open(path)['nominal']
        fb = f.arrays(variables_data, library="ak")
        fb['VertexBDTScore'] = fb['BDTScore'] # renaming BDTScore to ensure this is recognized as Vertex BDT Score
        
        fb = fb[ak.num(fb['ph_eta']) > 0]     # for abs(ak.firsts(fb['ph_eta'])) to have value to the reweighting
                
        mask1 = (ak.firsts(fb['ph_topoetcone40'])-2450.)/ak.firsts(fb['ph_pt']) > 0.1   # jet_faking_photon cut
        fb = fb[mask1]
        fb = fb[fb['n_ph_baseline'] == 1]

    else: # MC
        path = f"/data/tmathew/ntups/mc23d/{ntuple_name}_y.root" 
        path_BDT = f"/data/fpiazza/ggHyyd/Ntuples/MC23d/withVertexBDT/mc23d_{ntuple_name}_y_BDT_score.root" 
        print('processing file: ', path)
        f = uproot.open(path)['nominal']
        fb = f.arrays(variables, library="ak")

        # add BDT score to fb
        f_BDT = uproot.open(path_BDT)['nominal']
        fb_BDT = f_BDT.arrays(["event", "BDTScore"], library="ak")
        tmp = fb["event"] == fb_BDT["event"]
        if np.all(tmp) == True:
            fb["VertexBDTScore"] = fb_BDT["BDTScore"]
        else: 
            print("Something is wrong, need arranging")

        fb = fb[ak.num(fb['ph_eta']) > 0]     # for abs(ak.firsts(fb['ph_eta'])) to have value to the reweighting
        fb = fb[fb['n_ph'] == 1]
        
        # Zjets and Wjets (rule out everything except for e->gamma)
        if ntuple_name == 'Zjets' or ntuple_name == 'Wjets':
            mask = ak.firsts(fb['ph_truth_type']) == 2
            fb = fb[mask]
        
        # goodPV on signal only
        if ntuple_name == 'ggHyyd':
            fb = fb[ak.num(fb['pv_z']) > 0]
            good_pv_tmp = (np.abs(ak.firsts(fb['pv_truth_z']) - ak.firsts(fb['pv_z'])) <= 0.5)
            fb = fb[good_pv_tmp]

    print_cut(ntuple_name, fb, 'before cut')
    wcut.append(sum(getWeight(fb, ntuple_name)))

    fb = fb[fb['n_mu_baseline'] == 0]
    wcut.append(sum(getWeight(fb, ntuple_name)))
    fb = fb[fb['n_el_baseline'] == 0]
    wcut.append(sum(getWeight(fb, ntuple_name)))
    fb = fb[fb['n_tau_baseline'] == 0]
    wcut.append(sum(getWeight(fb, ntuple_name)))
    fb = fb[fb['trigger_HLT_g50_tight_xe40_cell_xe70_pfopufit_80mTAC_L1eEM26M']==1]
    wcut.append(sum(getWeight(fb, ntuple_name)))
    fb = fb[ak.num(fb['ph_pt']) > 0] # prevent none values in Tbranch
    fb = fb[ak.firsts(fb['ph_pt']) >= 50000] # ph_pt cut (basic cut)
    wcut.append(sum(getWeight(fb, ntuple_name)))
    fb = fb[fb['met_tst_et'] >= 100000] # MET cut (basic cut)
    wcut.append(sum(getWeight(fb, ntuple_name)))
    fb = fb[fb['n_jet_central'] <= 4] # n_jet_central cut (basic cut)
    wcut.append(sum(getWeight(fb, ntuple_name)))

    mt_tmp = np.sqrt(2 * fb['met_tst_et'] * ak.firsts(fb['ph_pt']) * 
                            (1 - np.cos(fb['met_tst_phi'] - ak.firsts(fb['ph_phi'])))) / 1000
    mask1 = mt_tmp >= 100 # trigger cut
    fb = fb[mask1]
    wcut.append(sum(getWeight(fb, ntuple_name)))

    print_cut(ntuple_name, fb, 'after basic cut')


    ucut.append(len(fb))

    unweighted_acut.append(ucut)
    weighted_acut.append(wcut)
    test(fb) # check for none value

    print(f"Reading Time for {ntuple_name}: {(time.time()-start_time)} seconds\n")



    tot.append(fb)

    fb = 0
    fb_BDT = 0
    tmp = 0


processing file:  /data/tmathew/ntups/mc23d/ggHyyd_y.root
Unweighted Events before cut:  86910
Weighted Events before cut:  8732.756
Unweighted Events after basic cut:  4587
Weighted Events after basic cut:  461.92218
Number of none values:  0
Reading Time for ggHyyd: 7.528102159500122 seconds

processing file:  /data/tmathew/ntups/mc23d/Zjets_y.root


: 

In [1]:
import psutil

# Get memory info
mem = psutil.virtual_memory()

print(f"Total RAM: {mem.total / (1024 ** 3):.2f} GB")
print(f"Available RAM: {mem.available / (1024 ** 3):.2f} GB")
print(f"Used RAM: {mem.used / (1024 ** 3):.2f} GB")
print(f"RAM Usage: {mem.percent}%")


Total RAM: 250.84 GB
Available RAM: 212.74 GB
Used RAM: 35.02 GB
RAM Usage: 15.2%


In [2]:
def getCutDict():
    cut_dict = {}
signal_name = 'ggHyyd'  # Define signal dataset
cut_name = 'basic'

def getCutDict():
    cut_dict = {}
    # Reduced Features
    cut_dict['VertexBDTScore'] = {
        'lowercut': np.arange(0, 0.3+0.01, 0.01) # VertexBDTScore > cut
    }
    cut_dict['balance'] = {
        'lowercut': np.arange(0, 1.5 + 0.01, 0.01), # balance > cut
        'uppercut': np.arange(1.5, 9, 0.05) # balance < cut
    }
    cut_dict['dmet'] = {
        'lowercut': np.arange(-30000, 10000 + 100, 100), # dmet > cut
        'uppercut': np.arange(10000, 100000 + 100, 100), # -10000 < dmet < cut
    }
    cut_dict['metsig'] = {
        'lowercut': np.arange(0, 10 + 1, 1), # metsig > cut
        'uppercut': np.arange(10, 30 + 1, 1), # metsig < cut 
    }
    cut_dict['jetterm'] = {
        'lowercut': np.arange(0, 150000+500, 500) # jetterm > cut
    }
    cut_dict['dphi_met_phterm'] = {
        'lowercut': np.arange(1, 2 + 0.01, 0.01), # dphi_met_phterm > cut
    }
    cut_dict['dphi_met_central_jet'] = {
        'lowercut': np.arange(1.5, 2.8, 0.01)
    }
    cut_dict['ph_eta'] = {
        'uppercut': np.arange(1, 2.5 + 0.01, 0.01), # ph_eta < cut
    }
    cut_dict['ph_pt'] = {
        'lowercut': np.arange(50000, 100000 + 1000, 1000),  # ph_pt > cut
        'uppercut': np.arange(100000, 300000 + 1000, 1000),  # ph_pt > cut
    }

    # Other Features
    cut_dict['dphi_jj'] = {
        'uppercut': np.arange(1, 3.14 + 0.01, 0.01) # dphi_jj < cut
    }
    cut_dict['dphi_phterm_jetterm'] = {
        'lowercut': np.arange(1, 2.5 + 0.05, 0.05), # dphi_phterm_jetterm > cut
        'uppercut': np.arange(2, 4 + 0.1, 0.1) # dphi_phterm_jetterm < cut
    }
    cut_dict['jet_central_eta'] = {
        'lowercut': np.arange(-2.5, 0+0.01, 0.01), # jet_central_eta > cut
        'uppercut': np.arange(0, 2.5+0.01, 0.01) # jet_central_eta < cut
    }
    cut_dict['jet_central_pt2'] = {
        'lowercut': np.arange(20000, 100000+1000, 1000) # jet_central_pt2 > cut
    }
    cut_dict['metsigres'] = {
        'lowercut': np.arange(8600, 15000, 100),
        'uppercut': np.arange(12000, 60000, 100)
    }
    cut_dict['met_noJVT'] = {
        'lowercut': np.arange(50000, 120000, 100),
        'uppercut': np.arange(100000, 250000, 100)
    }
    cut_dict['softerm'] = {
        'uppercut': np.arange(10000, 40000, 100)
    }
    cut_dict['n_jet_central'] = {
        'uppercut': np.arange(0, 8+1, 1) # njet < cut
    }

    return cut_dict
cut_config = getCutDict()

In [3]:
%%time
signal_name='ggHyyd'
initial_cut = []
tot2 = tot

# < -- Initial Cut on all variables (maximize the significance * acceptance) -- > 
for cut_var, cut_types in cut_config.items():
    for cut_type, cut_values in cut_types.items():
        sig_simple_list, sigacc_simple_list, acceptance_values = calculate_significance(
            cut_var, cut_type, cut_values, tot2, ntuple_names, signal_name, getVarDict, getWeight
        )

        best_cut, best_sig, idx = get_best_cut(cut_values, sigacc_simple_list) 
        
        if idx == 0 or idx == len(sigacc_simple_list) - 1: # I chose to use index to indicate not to make unnecessary cut (for initial cut)
            print(cut_var, idx, len(sigacc_simple_list))
            continue
            
        result = {
            "cut_var": cut_var,
            "cut_type": cut_type,
            "best_cut": best_cut,
            "best_sig_x_acc": best_sig,
            "significance": sig_simple_list[idx],
            "acceptance": acceptance_values[idx]
        }

        print(result)
        initial_cut.append(dict(list(result.items())[:3]))

{'cut_var': 'VertexBDTScore', 'cut_type': 'lowercut', 'best_cut': 0.09, 'best_sig_x_acc': 0.5300952216935515, 'significance': 0.6397698893038751, 'acceptance': 82.85716951611163}
{'cut_var': 'balance', 'cut_type': 'lowercut', 'best_cut': 0.3, 'best_sig_x_acc': 0.3680869481323753, 'significance': 0.36821775624246844, 'acceptance': 99.96447533887883}
{'cut_var': 'balance', 'cut_type': 'uppercut', 'best_cut': 1.8000000000000003, 'best_sig_x_acc': 0.46134525047490643, 'significance': 0.5780321697227262, 'acceptance': 79.81307523008748}
{'cut_var': 'dmet', 'cut_type': 'lowercut', 'best_cut': -16600, 'best_sig_x_acc': 0.4129232806067522, 'significance': 0.43106329002235677, 'acceptance': 95.79179906164968}
{'cut_var': 'dmet', 'cut_type': 'uppercut', 'best_cut': 48900, 'best_sig_x_acc': 0.3275416905797266, 'significance': 0.32754170160018753, 'acceptance': 99.99999663540218}
{'cut_var': 'metsig', 'cut_type': 'lowercut', 'best_cut': 4, 'best_sig_x_acc': 0.361979872020551, 'significance': 0.376

In [4]:
tot2_initial_cut = apply_all_cuts(tot2, ntuple_names, initial_cut, getVarDict)
final_significance = compute_total_significance(tot2_initial_cut, ntuple_names, signal_name, getVarDict, getWeight)
print('after initial cutting, signficance: ', final_significance)

after initial cutting, signficance:  1.8098839816515948


In [5]:
%%time
# < -- n-1 iterations until no further improvement (max significance) -- >
optimized_cuts, final_significance = n_minus_1_optimizer(
    initial_cut, cut_config, tot2, ntuple_names, signal_name, getVarDict, getWeight, final_significance
)
print('after optimized cutting, signficance: ', final_significance)



--- Iteration 1 ---
Updating VertexBDTScore (lowercut): 0.09 → 0.12  (sig 1.81 → 1.81)
Updating balance (lowercut): 0.3 → 0.72  (sig 1.81 → 1.88)
Updating balance (uppercut): 1.8000000000000003 → 5.2500000000000036  (sig 1.88 → 1.90)
Updating dmet (lowercut): -16600 → -19500  (sig 1.90 → 1.90)
Updating metsig (lowercut): 4 → 5  (sig 1.90 → 1.95)
Updating metsig (uppercut): 16 → 12  (sig 1.95 → 1.99)
Updating jetterm (lowercut): 112500 → 81500  (sig 1.99 → 2.06)
Updating dphi_met_phterm (lowercut): 1.2100000000000002 → 1.4300000000000004  (sig 2.06 → 2.14)
Updating dphi_met_central_jet (lowercut): 1.7800000000000002 → 2.0800000000000005  (sig 2.14 → 2.19)
Updating ph_eta (uppercut): 2.370000000000001 → 1.6500000000000006  (sig 2.19 → 2.31)
Updating dphi_jj (uppercut): 2.7600000000000016 → 3.140000000000002  (sig 2.31 → 2.33)
Updating dphi_phterm_jetterm (lowercut): 1.4500000000000004 → 1.7500000000000007  (sig 2.33 → 2.34)
Updating metsigres (lowercut): 10300 → 10700  (sig 2.34 → 2.34)

In [6]:
print( ' < -- Final Optimized Cuts -- > ')
# print(optimized_cuts)

for cut in optimized_cuts:
    var = cut['cut_var']
    val = cut['best_cut']
    if cut['cut_type'] == 'uppercut':
        print(f"{var} <= {val}")
    elif cut['cut_type'] == 'lowercut':
        print(f"{var} >= {val}")
        
print('after optimized cutting, signficance: ', final_significance)


 < -- Final Optimized Cuts -- > 
VertexBDTScore >= 0.11
balance >= 0.8
balance <= 4.000000000000002
dmet >= -19100
dmet <= 48900
metsig >= 5
metsig <= 12
jetterm >= 86500
dphi_met_phterm >= 1.5400000000000005
dphi_met_central_jet >= 2.0500000000000007
ph_eta <= 1.6500000000000006
dphi_jj <= 3.140000000000002
dphi_phterm_jetterm >= 1.8500000000000008
dphi_phterm_jetterm <= 3.000000000000001
metsigres >= 10400
metsigres <= 33600
met_noJVT >= 102600
met_noJVT <= 242400
n_jet_central <= 4
after optimized cutting, signficance:  2.4479681389997188


In [7]:
tot2_optimized_cuts = apply_all_cuts(tot2, ntuple_names, optimized_cuts, getVarDict)

In [8]:
tot2_optimized_cuts

[<Array [{actualIntPerXing: 56.5, ...}, ...] type='1157 * ?{actualIntPerXing...'>,
 <Array [{actualIntPerXing: 56.5, ...}, ...] type='30 * ?{actualIntPerXing: ...'>,
 <Array [{actualIntPerXing: 50.5, ...}, ...] type='8575 * ?{actualIntPerXing...'>,
 <Array [{actualIntPerXing: 64.5, ...}, ...] type='6605 * ?{actualIntPerXing...'>,
 <Array [{actualIntPerXing: 66.5, ...}, ...] type='2209 * ?{actualIntPerXing...'>,
 <Array [{actualIntPerXing: 51.5, ...}, ...] type='420 * ?{actualIntPerXing:...'>,
 <Array [{actualIntPerXing: 57.6, ...}, ...] type='30 * ?{actualIntPerXing: ...'>]

In [17]:
print('< -- Sum of weight each process -- >')

for i in range(len(tot2_optimized_cuts)):
    print(ntuple_names[i], sum(getWeight(tot2_optimized_cuts[i], ntuple_names[i])))

< -- Sum of weight each process -- >
ggHyyd 117.65650455146829
Zjets 2.3715278876334196
Zgamma 382.2246941178605
Wgamma 848.8890796693562
Wjets 556.5861277854304
gammajet_direct 202.48297826768368
data23 317.4929659454739


In [14]:
# < -- Save data after cuts to a csv file for BDT input -- >
Vars = [
    'balance', 
    'VertexBDTScore',
    'dmet',
    'dphi_jj',
    'dphi_met_central_jet',
    'dphi_met_phterm',
    'dphi_met_ph',
    'dphi_met_jetterm',
    'dphi_phterm_jetterm',
    'dphi_ph_centraljet1',
    'ph_pt',
    'ph_eta',
    'ph_phi',
    'jet_central_eta',
    'jet_central_pt1',
    'jet_central_pt2',
    'jetterm',
    'jetterm_sumet',
    'metsig',
    'metsigres',
    'met',
    'met_noJVT',
    'metplusph',
    'failJVT_jet_pt1',
    'softerm',
    'n_jet_central'
]

data_list = []

for j in range(len(ntuple_names)):
    process = ntuple_names[j]
    fb = tot2_optimized_cuts[j] 
    
    data_dict = {}
    
    for var in Vars:
        var_config = getVarDict(fb, process, var_name=var)
        data_dict[var] = var_config[var]['var']
    
    weights = getWeight(fb, process)
    data_dict['weights'] = weights
    
    n_events = len(weights)
    data_dict['process'] = [process] * n_events
    label = 1 if process == 'ggHyyd' else 0
    data_dict['label'] = [label] * n_events
    
    df_temp = pd.DataFrame(data_dict)
    data_list.append(df_temp)

df_all = pd.concat(data_list, ignore_index=True)
df_all.head()

df_all.to_csv("/data/jlai/ntups/csv/jet_faking_BDT_input_basic_reduced2.csv", index=False)